# SUMO Brussels — Visualization Notebook

This notebook is prepared for analysing and visulizating SUMO outputs from the experimental Brussels traffic scenarios.

In [ ]:
# SUMO Brussels — Visualization Notebook
# Repository: https://github.com/aed90/SUMO-Brussels-Visualization-Notebook

"""
This notebook provides:
- A reproducible walkthrough to load any SUMO repo, detect networks and sumo output types within it.
- All necessary details to understand the simulation outputs meaningfully with summaries, plots, and map graphs.
- Uses SUMO's visualization tools from <SUMO_HOME>/tools/visualization.
- Contains both command-line usages and python code for custom plots.
- Users can adjust various arguments, such as the target simulation repository, the repository structure, and the local output directory, and thresholds for any other simulation repository testing.
- If a sumo output type is not found, the notebook does not fail and simply skips it.

The tool provides:

1. Statistical Summary: Aggregated performance metrics and stats.
2. Operational Checks: Vehicle movement rationality.
3. Detector Validation: Sensor functionality.
4. Network Analysis: Flow and congestion patterns.
5. Scenario Comparison: Real vs augmented model performance.


NOTE: Additional analysis will be included in time. 
"""

# ---------------------------------------------
# Section 0 — Install / pre-checks
# ---------------------------------------------

# Recommended environment: conda or venv with Python 3.9+ and these packages installed.
# Python packages used in this notebook (uncomment to install in a notebook environment):
# !pip install pandas geopandas matplotlib numpy sumolib

import os, sys, subprocess, shutil
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

try:
    import geopandas as gpd
except Exception:
    gpd = None

try:
    from sumolib import net as sumonet
    from sumolib import routes as sumoroutes
except Exception:
    sumonet = None

# Create plots directory - replace with your own path
plots_dir = Path('/home/adingil/test/plots')
plots_dir.mkdir(parents=True, exist_ok=True)
print(f"Plots will be saved to: {plots_dir.absolute()}")

# ---------------------------------------------
# Section 1 — Clone any repo and file discovery
# ---------------------------------------------

# EXAMPLE REPO PROVIDING EXPERIMENTAL BRUSSELS SCENARIOS

REPO_URL = 'https://gitlab.com/traffic-sim/traffic_data_augmentation.git'
LOCAL_DIR = Path('/home/adingil/test/repo') #replace with your own path
if not LOCAL_DIR.exists():
    print('Cloning repo...')
    subprocess.run(['git', 'clone', REPO_URL, str(LOCAL_DIR)], check=True)
else:
    print('Repo already present at', LOCAL_DIR)

# run git lfs pull if needed
print('\nChecking for large files...')
subprocess.run(['git', '-C', str(LOCAL_DIR), 'lfs', 'pull'], capture_output=True)

# Detect candidate paths
scenarios_dir = LOCAL_DIR / 'scenarios'
dataset_dir = LOCAL_DIR / 'dataset' / 'augmented_data'

# Use the repo structure instead of external directories
outputs_real_dir = LOCAL_DIR / 'real_outputs'
outputs_augmented_dir = LOCAL_DIR / 'augmented_outputs'
maps_dir = LOCAL_DIR / 'scenarios' / 'map'

print('scenarios exists?', scenarios_dir.exists())
print('dataset exists?', dataset_dir.exists())
print('maps exists?', maps_dir.exists())
print('real outputs exists?', outputs_real_dir.exists())
print('augmented outputs exists?', outputs_augmented_dir.exists())

# ---------------------------------------------
# Section 2 — Locate SUMO tools and visualization scripts
# ---------------------------------------------

SUMO_BIN = shutil.which('sumo') or shutil.which('sumo-gui')
SUMO_GUI = shutil.which('sumo-gui')
PLOT_TOOLS_DIR = os.getenv('SUMO_HOME') and Path(os.getenv('SUMO_HOME')) / 'tools' / 'visualization'

print('SUMO binary:', SUMO_BIN)
print('SUMO-GUI:', SUMO_GUI)
print('Visualization tools dir:', PLOT_TOOLS_DIR if PLOT_TOOLS_DIR else 'SUMO_HOME not set — try to call installed scripts directly')

# ---------------------------------------------
# Section 3 — Detect sumo network and sumo outputs in the repo
# ---------------------------------------------

net_xml = None
if maps_dir.exists():
    for p in maps_dir.glob('*'):
        if p.suffix == '.xml' and 'net' in p.name:
            net_xml = p
            break
print('Detected network xml:', net_xml)

#Use the repo directories
real_output_files = (list(outputs_real_dir.rglob('*edgedata.out*.xml')) + 
                list(outputs_real_dir.rglob('*detector.out*.xml')) + 
                list(outputs_real_dir.rglob('*fcd.out*.xml')) + 
                list(outputs_real_dir.rglob('*tripinfo.out*.xml')) + 
                list(outputs_real_dir.rglob('*summary.out*.xml')) + 
                list(outputs_real_dir.rglob('*vehroute.out*.xml')))
print('Found real SUMO output candidates:', [p.name for p in real_output_files[:10]])
print('Total real output files:', len(real_output_files))

augmented_output_files = (list(outputs_augmented_dir.rglob('*edgedata.out*.xml')) + 
                list(outputs_augmented_dir.rglob('*detector.out*.xml')) + 
                list(outputs_augmented_dir.rglob('*fcd*.xml')) + 
                list(outputs_augmented_dir.rglob('*tripinfo.out*.xml')) + 
                list(outputs_augmented_dir.rglob('*summary.out*.xml')) + 
                list(outputs_augmented_dir.rglob('*vehroute.out*.xml')))
print('Found augmented SUMO output candidates:', [p.name for p in augmented_output_files[:10]])
print('Total augmented output files:', len(augmented_output_files))

# If no files found in expected locations, try alternative locations
if len(real_output_files) == 0:
    print("No files found in real_outputs directory, trying alternative locations...")
    # Try to find any output files in the entire repo
    alternative_real_files = list(LOCAL_DIR.rglob('*edgedata*.xml')) + list(LOCAL_DIR.rglob('*detector*.xml'))
    print(f"Found alternative files: {[f.name for f in alternative_real_files[:5]]}")
    real_output_files = alternative_real_files

if len(augmented_output_files) == 0:
    print("No files found in augmented_outputs directory, trying alternative locations...")
    alternative_aug_files = list(LOCAL_DIR.rglob('*tripinfo*.xml')) + list(LOCAL_DIR.rglob('*summary*.xml'))
    print(f"Found alternative files: {[f.name for f in alternative_aug_files[:5]]}")
    augmented_output_files = alternative_aug_files

# ---------------------------------------------
# Section 4 — SUMO visualization PART1
# ---------------------------------------------

# For generating a series of 2D plots comparing two selected output attributes from one or more XML output files, grouped by a third attribute when needed.
# Simulation models are functioning correctly when the vehicle movement metrics show rational patterns. This section allows you to control the arrival, departure, running, waiting, halting, and some other patterns of the vehicles to ensure this.
# Detectors are functional if occupancy and speed parameters show a rational variation. This section allows you to pick important loop detectors and investigate their operational metrics during simulation time.

#Extract:

def extract_date_from_path(file_path):
    """Extract date from file path like .../real_outputs/2024-03-25/3000/idExp0/calibration_iter9/..."""
    try:
        parts = file_path.parts
        # Look for the date pattern in the path
        for i, part in enumerate(parts):
            if '-' in part and len(part) == 10:  # Date pattern: YYYY-MM-DD
                # Check if it's actually a date
                year, month, day = part.split('-')
                if year.isdigit() and month.isdigit() and day.isdigit():
                    return part
        # If no date found in standard location, try to find any date-like pattern
        for part in parts:
            if '-' in part and len(part) == 10:
                year, month, day = part.split('-')
                if year.isdigit() and month.isdigit() and day.isdigit():
                    return part
        return "unknown_date"
    except:
        return "unknown_date"
        
# Visualization basement:

def run_plot_xml_attributes(xmlfile, xattr='begin', yattr='nVehEntered', idattr='id', out='plot.png', extra_args=None):
    """Call SUMO's plotXMLAttributes.py script if available in PATH or SUMO_HOME/tools/visualization."""
    # Check if input file exists
    if not xmlfile.exists():
        print(f"Input file {xmlfile} not found. Skipping.")
        return None
        
    script = shutil.which('plotXMLAttributes.py')
    if not script and PLOT_TOOLS_DIR:
        candidate = PLOT_TOOLS_DIR / 'plotXMLAttributes.py'
        if candidate.exists():
            script = str(candidate)
    if not script:
        print('plotXMLAttributes.py not found. Please set SUMO_HOME or install SUMO tools. Falling back to Python plotting for', xmlfile)
        return None
        
    cmd = [script, '-x', xattr, '-y', yattr, '-i', idattr, str(xmlfile), '-o', out]
    if extra_args:
        cmd += extra_args
    print('Running:', ' '.join(cmd))
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
        if result.returncode != 0:
            print(f"Error running plotXMLAttributes: {result.stderr}")
            return None
        print(f"Successfully created plot: {out}")
        return out
    except subprocess.TimeoutExpired:
        print(f"Plotting timed out for {xmlfile}")
        return None
    except Exception as e:
        print(f"Unexpected error plotting {xmlfile}: {e}")
        return None

print("\n" + "="*60)
print("Section 4 - Generating Operational Plots")
print("="*60)

# Example-1 General overview plot of vehicle dynamics over time, observing running & halting vehicles evolution over simulation time:

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'summary' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'veh_runninghalting_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='time',
                yattr='running,halting',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Running & Halting Vehicles [s]',
                    '--title', 'Running & Halting Vehicles over Time',
                    '--titlesize', '16'
                ]
            )

# Example 1.1 General overview plot of vehicle dynamics over time, observing running & waiting vehicles evolution during simulations.

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'summary' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'veh_runningwaiting_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='time',
                yattr='running,waiting',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Running & Waiting Vehicles [s]',
                    '--title', 'Running & Waiting Vehicles over Time',
                    '--titlesize', '16'
                ]
            )

# Example 1.2 General overview plot of vehicle dynamics over time, observing running & teleported vehicles evolution during simulations.

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'summary' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'veh_runningteleport_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='time',
                yattr='running,teleports',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Running & Teleported Vehicles [s]',
                    '--title', 'Running & Teleported Vehicles over Time',
                    '--titlesize', '16'
                ]
            )

# Example 1.3 General overview plot of vehicle dynamics over time, observing mean travel time & mean waiting time evolution during simulations.

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'summary' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'veh_meanTTWT_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='time',
                yattr='meanTravelTime,meanWaitingTime',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Travel Time & Waiting Time [s]',
                    '--title', 'Travel and Waiting Time Evolution of Vehicles',
                    '--titlesize', '16'
                ]
            )

    
# Example-2 General overview plot of departure and arrival timing during day, investigating departure-arrival evolution over simulation time:

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'vehroute' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'depart_arrival_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='depart',
                yattr='arrival',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--scatterplot',
                    '--xlabel', 'Depart time [s]',
                    '--ylabel', 'Arrival time [s]',
                    '--title', 'Departure time over arrival time',
                    '--titlesize', '16'
                ]
            )

# Example-3 General overview plot of departure delay, investigating delays over simulation time:

if len(real_output_files) > 0:
    for f in real_output_files:
        if 'tripinfo' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'depart_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='depart',
                yattr='departDelay',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--scatterplot',
                    '--xlabel', 'Depart time [s]',
                    '--ylabel', 'Depart delay [s]',
                    '--ylim', '0,1800',
                    '--xticks', '0,86400,7200,10',
                    '--yticks', '0,1800,100,10', 
                    '--xgrid',
                    '--ygrid',
                    '--title', 'Depart delay over depart time',
                    '--titlesize', '16'
                ]
            )

# Example-4 General overview plot of detector occupancy variations over time, investigating important detectors’ patterns:
if len(real_output_files) > 0:
    for f in real_output_files:
        if 'detector.out' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'detector_occupancy_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='begin',
                yattr='occupancy',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--filter-ids', 'BXLBXL034144F1_0,BXLJET127338F1_0,BXLUCC034671F1_0,BXLUCC127337F1_0,BXLJET139438B1_0', #change with detector IDs you would like to investigate
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Occupancy',
                    '--ylim', '0,50',
                    '--xticks', '0,86400,7200,10',
                    '--yticks', '0,50,5,10',
                    '--xgrid',
                    '--ygrid',
                    '--title', 'Detector Occupancy',
                    '--titlesize', '16'
                ]
            )

# Example-5 General overview plot of vehicle speed variation over detectors, investigating important detectors’ pattern:
if len(real_output_files) > 0:
    for f in real_output_files:
        if 'detector.out' in f.name:
            date_str = extract_date_from_path(f)
            print(f'Will attempt to plot Data from {f.name} (Date: {date_str})')
            output_path = plots_dir / f'detector_speed_{date_str}_{f.stem}.png'
            run_plot_xml_attributes(
                f,
                xattr='begin',
                yattr='speed',
                idattr='id',
                out=str(output_path),
                extra_args=[
                    '--legend',
                    '--filter-ids', 'BXLBXL034144F1_0,BXLJET127338F1_0,BXLUCC034671F1_0,BXLUCC127337F1_0,BXLJET139438B1_0', #change with detector IDs you would like to investigate
                    '--xlabel', 'Time [s]',
                    '--ylabel', 'Speed',  #m/s
                    '--ylim', '0,16',
                    '--xticks', '0,86400,7200,10',
                    '--yticks', '0,16,2,10',
                    '--xgrid',
                    '--ygrid',
                    '--xgrid',                    
                    '--title', 'Speed over Detectors',
                    '--titlesize', '16',
                ]
            )
else:
    print("No real output files found for Section 4 analysis")

# ---------------------------------------------
# Section 5 — SUMO Visualization PART2
# ---------------------------------------------

# Visualize traffic & simulation outputs
# Urban traffic is rational if its patterns reflect real-life conditions. This section allows you to investigate important traffic metrics during simulation time.
# For generating a network visualization where the color and width of the edges are dynamically styled based on predefined edge attributes. 

# This section also provides all necessary performance metrics for each edge in the road network for every scenario in CSV format for users and outputs descriptive stats of these metrics (min, max, mean, median, etc.) for each simulation, as well as global statistics to help explore thresholds and critical values for better visualization.

# Pop up some descriptive stats (min, max, mean, median, etc.) of important traffic values from the simulation outputs to explore the results.
 

def analyze_edge_statistics(file_path, global_stats=None):
    """Analyze max, min, mean, median of edge attributes."""
    if not file_path.exists():
        print(f"Edge data file {file_path} not found. Skipping statistics.")
        return None
        
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        indicators = [
            "occupancy", "timeLoss", "density", "waitingTime", "traveltime", "speed", "speedRelative",
            "overlapTraveltime", "departed", "arrived", "entered", "teleported", "sampledSeconds"
        ]

        stats = {
            key: {
                'max': {'id': None, 'value': float('-inf'), 'file': None, 'date': None},  
                'min': {'id': None, 'value': float('inf'), 'file': None, 'date': None},  
                'sum': 0.0,
                'count': 0,
                'values': []
            } for key in indicators
        }

        # Extract date once for this file
        date_str = extract_date_from_path(file_path)

        for edge in root.iter('edge'):
            edge_id = edge.attrib['id']
            for indicator in indicators:
                value_str = edge.attrib.get(indicator)
                if value_str is not None:
                    try:
                        value = float(value_str)
                        # max
                        if value > stats[indicator]['max']['value']:
                            stats[indicator]['max'] = {
                                'id': edge_id, 
                                'value': value, 
                                'file': file_path.name,
                                'date': date_str  
                            }
                        # min
                        if value < stats[indicator]['min']['value']:
                            stats[indicator]['min'] = {
                                'id': edge_id, 
                                'value': value, 
                                'file': file_path.name,
                                'date': date_str  
                            }
                        stats[indicator]['sum'] += value
                        stats[indicator]['count'] += 1
                        stats[indicator]['values'].append(value)
                        
                        # global statistics if provided
                        if global_stats is not None:
                            if indicator not in global_stats:
                                global_stats[indicator] = {
                                    'global_max': {'id': None, 'value': float('-inf'), 'file': None, 'date': None},
                                    'global_min': {'id': None, 'value': float('inf'), 'file': None, 'date': None},
                                    'global_sum': 0.0,
                                    'global_count': 0,
                                    'global_values': []
                                }
                            
                            # global max
                            if value > global_stats[indicator]['global_max']['value']:
                                global_stats[indicator]['global_max'] = {
                                    'id': edge_id, 
                                    'value': value, 
                                    'file': file_path.name,
                                    'date': date_str  
                                }
                            # global min
                            if value < global_stats[indicator]['global_min']['value']:
                                global_stats[indicator]['global_min'] = {
                                    'id': edge_id, 
                                    'value': value, 
                                    'file': file_path.name,
                                    'date': date_str  
                                }
                            global_stats[indicator]['global_sum'] += value
                            global_stats[indicator]['global_count'] += 1
                            global_stats[indicator]['global_values'].append(value)
                            
                    except (ValueError, TypeError):
                        continue

        # Print results for this file
        print(f"\n{'='*60}")
        print(f"Edge Statistics for {file_path.name} (Date: {date_str})")
        print(f"{'='*60}")
        
        for indicator, data in stats.items():
            if data['count'] > 0:
                avg = data['sum'] / data['count']
                sorted_values = sorted(data['values'])
                n = len(sorted_values)
                if n % 2 == 0:
                    median = (sorted_values[n//2 - 1] + sorted_values[n//2]) / 2
                else:
                    median = sorted_values[n//2]
                
                print(f"\nIndicator: {indicator}")
                print(f"  Count:  {data['count']} edges")
                print(f"  Max:    {data['max']['value']:.6f} on edge {data['max']['id']} (Date: {data['max']['date']})")  # Show date
                print(f"  Min:    {data['min']['value']:.6f} on edge {data['min']['id']} (Date: {data['min']['date']})")  # Show date
                print(f"  Mean:   {avg:.6f}")
                print(f"  Median: {median:.6f}")
            else:
                print(f"\nIndicator: {indicator} - No data found")
                
        return stats
        
    except Exception as e:
        print(f"Error analyzing edge statistics for {file_path}: {e}")
        return None

def print_global_statistics(global_stats):
    """Print global statistics across all files."""
    if not global_stats:
        print("No global statistics to display.")
        return
        
    print(f"\n{'='*80}")
    print("GLOBAL STATISTICS (Across All Files)")
    print(f"{'='*80}")
    
    for indicator, data in global_stats.items():
        if data['global_count'] > 0:
            global_avg = data['global_sum'] / data['global_count']
            # Calculate global median
            sorted_global_values = sorted(data['global_values'])
            n_global = len(sorted_global_values)
            if n_global % 2 == 0:
                global_median = (sorted_global_values[n_global//2 - 1] + sorted_global_values[n_global//2]) / 2
            else:
                global_median = sorted_global_values[n_global//2]
            
            print(f"\nIndicator: {indicator}")
            print(f"  Global Count:  {data['global_count']} edges across all files")
            print(f"  Global Max:    {data['global_max']['value']:.6f} on edge {data['global_max']['id']} (File: {data['global_max']['file']}, Date: {data['global_max']['date']})")
            print(f"  Global Min:    {data['global_min']['value']:.6f} on edge {data['global_min']['id']} (File: {data['global_min']['file']}, Date: {data['global_min']['date']})")
            print(f"  Global Mean:   {global_avg:.6f}")
            print(f"  Global Median: {global_median:.6f}")
        else:
            print(f"\nIndicator: {indicator} - No global data found")

def save_global_statistics_to_csv(global_stats, file_path):
    """Save global statistics to a CSV file."""
    try:
        # Prepare data for CSV
        csv_data = []
        for indicator, data in global_stats.items():
            if data['global_count'] > 0:
                global_avg = data['global_sum'] / data['global_count']
                # Calculate global median
                sorted_global_values = sorted(data['global_values'])
                n_global = len(sorted_global_values)
                if n_global % 2 == 0:
                    global_median = (sorted_global_values[n_global//2 - 1] + sorted_global_values[n_global//2]) / 2
                else:
                    global_median = sorted_global_values[n_global//2]
                
                csv_data.append({
                    'Indicator': indicator,
                    'Global_Count': data['global_count'],
                    'Global_Max_Value': data['global_max']['value'],
                    'Global_Max_Edge': data['global_max']['id'],
                    'Global_Max_File': data['global_max']['file'],
                    'Global_Max_Date': data['global_max']['date'], 
                    'Global_Min_Value': data['global_min']['value'],
                    'Global_Min_Edge': data['global_min']['id'],
                    'Global_Min_File': data['global_min']['file'],
                    'Global_Min_Date': data['global_min']['date'], 
                    'Global_Mean': global_avg,
                    'Global_Median': global_median
                })
        
        if csv_data:
            df = pd.DataFrame(csv_data)
            df.to_csv(file_path, index=False)
            print(f"Global statistics saved to CSV: {file_path}")
            return True
        else:
            print("No global statistics data to save.")
            return False
            
    except Exception as e:
        print(f"Error saving global statistics to CSV: {e}")
        return False

def save_individual_statistics_to_csv(individual_stats, plots_dir):
    """Save individual file statistics to a CSV file."""
    try:
        # Prepare data for CSV
        csv_data = []
        
        for file_path, stats in individual_stats.items():
            if stats is None:
                continue
                
            date_str = extract_date_from_path(file_path)
            
            for indicator, data in stats.items():
                if data['count'] > 0:
                    avg = data['sum'] / data['count']
                    sorted_values = sorted(data['values'])
                    n = len(sorted_values)
                    if n % 2 == 0:
                        median = (sorted_values[n//2 - 1] + sorted_values[n//2]) / 2
                    else:
                        median = sorted_values[n//2]
                    
                    csv_data.append({
                        'File': file_path.name,
                        'Date': date_str,
                        'Indicator': indicator,
                        'Count': data['count'],
                        'Max_Value': data['max']['value'],
                        'Max_Edge': data['max']['id'],
                        'Min_Value': data['min']['value'],
                        'Min_Edge': data['min']['id'],
                        'Mean': avg,
                        'Median': median
                    })
        
        if csv_data:
            df = pd.DataFrame(csv_data)
            csv_file_path = plots_dir / 'individual_statistics.csv'
            df.to_csv(csv_file_path, index=False)
            print(f"Individual statistics saved to CSV: {csv_file_path}")
            return True
        else:
            print("No individual statistics data to save.")
            return False
            
    except Exception as e:
        print(f"Error saving individual statistics to CSV: {e}")
        return False

def save_detailed_edge_data_to_csv(file_path, plots_dir):
    """Save detailed edge-level data to CSV for further analysis."""
    if not file_path.exists():
        print(f"File {file_path} not found. Skipping detailed edge data export.")
        return False
        
    try:
        # Extract date for unique filename
        date_str = extract_date_from_path(file_path)
        
        # Load and parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Indicators to extract
        indicators = [
            "occupancy", "timeLoss", "density", "waitingTime", "traveltime", "speed", "speedRelative",
            "overlapTraveltime", "departed", "arrived", "entered", "teleported", "sampledSeconds"
        ]

        # Collect all edge data
        edge_data = []
        for edge in root.iter('edge'):
            edge_info = {'Edge_ID': edge.attrib['id']}
            for indicator in indicators:
                value_str = edge.attrib.get(indicator)
                if value_str is not None:
                    try:
                        edge_info[indicator] = float(value_str)
                    except (ValueError, TypeError):
                        edge_info[indicator] = None
                else:
                    edge_info[indicator] = None
            edge_data.append(edge_info)
        
        if edge_data:
            df = pd.DataFrame(edge_data)
            csv_filename = f"detailed_edges_{date_str}_{file_path.stem}.csv"
            csv_file_path = plots_dir / csv_filename
            df.to_csv(csv_file_path, index=False)
            print(f"Detailed edge data saved to CSV: {csv_file_path}")
            return True
        else:
            print(f"No edge data found in {file_path.name}")
            return False
            
    except Exception as e:
        print(f"Error saving detailed edge data for {file_path} to CSV: {e}")
        return False

# Visualization basement:

def run_plot_net_dump(netfile, edgefile, measures, out, extra_args=None):
    """Call SUMO's plot_net_dump.py with given parameters."""
    # Check if input files exist
    if not netfile or not netfile.exists():
        print(f"Network file {netfile} not found. Skipping.")
        return None
    if not edgefile.exists():
        print(f"Edge data file {edgefile} not found. Skipping.")
        return None
        
    script = shutil.which('plot_net_dump.py')
    if not script and PLOT_TOOLS_DIR:
        candidate = PLOT_TOOLS_DIR / 'plot_net_dump.py'
        if candidate.exists():
            script = str(candidate)
    if not script:
        print('plot_net_dump.py not found. Please set SUMO_HOME or install SUMO tools.')
        return None
    
    cmd = [script, "-v", "-n", str(netfile), "--measures", measures, "-i", f"{edgefile},{edgefile}", "--output", out]
    if extra_args:
        cmd += extra_args
    print("Running:", " ".join(cmd))
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)  # Longer timeout for network plots
        if result.returncode != 0:
            print(f"Error running plot_net_dump: {result.stderr}")
            return None
        return out
    except subprocess.TimeoutExpired:
        print(f"Network plotting timed out for {edgefile}")
        return None
    except Exception as e:
        print(f"Unexpected error plotting network for {edgefile}: {e}")
        return None

# Analyzing Road Statistics and Generating Network Visualizations

print("\n" + "="*60)
print("Section 5 - Analyzing Road Statistics and Generating Network Visualizations")
print("="*60)

# Initialize global statistics dictionary
global_stats = {}
individual_stats = {}  # Dictionary to store individual file stats

for f in real_output_files:
    if 'edgedata.out' in f.name:
        file_stats = analyze_edge_statistics(f, global_stats)
        individual_stats[f] = file_stats  # Store individual file stats
        
        # Save detailed edge data for each file
        save_detailed_edge_data_to_csv(f, plots_dir)

# Print global statistics after processing all files
print_global_statistics(global_stats)

# Save statistics to CSV files
if global_stats:
    # Save global statistics
    global_csv_path = plots_dir / 'global_statistics.csv'
    save_global_statistics_to_csv(global_stats, global_csv_path)
    
    # Save individual file statistics
    save_individual_statistics_to_csv(individual_stats, plots_dir)


# To analyze edge indicators by identifying locations where their values peak or trough - observing their distribution across the transport system

for f in real_output_files:
    if 'edgedata.out' in f.name:
        date_str = extract_date_from_path(f)
        print(f'Generating network plots for {f.name} (Date: {date_str})')
        
# Example-1: A coloured city map providing Edge Usage Analysis - The number of vehicles that used each road during the simulation, ranked from the busiest to the least utilized streets. This data can be validated against real-world observations from sources like TomTom and Google to assess the model's accuracy.
        
        output_path = plots_dir / f'edgeusage_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "entered,entered",
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "0",
                "--max-color-value", "2000",
                "--max-width-value", "2000",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )

# Example-2: A coloured city map providing Overall Time Loss Analysis - The total delay in seconds (summed per road) caused by vehicles traveling slower than their desired speed. This is a primary method for detecting congestion and deadlocks; high values indicate a problematic model or network.

        output_path = plots_dir / f'timeloss_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "timeLoss,timeLoss",
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--max-color-value", "1800", # half hour
                "--max-width-value", "1800", # half hour
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )

# Example-3: A coloured city map providing Waiting Time Analysis - The total number of seconds (aggregated for each road) that vehicles spent halted (speed < speed threshold). This analysis helps identify intersections and road segments where traffic flow is inefficient, providing key insights into the model's performance.


        output_path = plots_dir / f'waitingtime_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "waitingTime,waitingTime",
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "0",
                "--max-color-value", "900", # 15 min
                "--max-width-value", "900", # 15 min
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )

# Example-4: Speed analysis - The mean speed (aggregated) on the roads during simulation.
# Better understand the functionality of the running model

        output_path = plots_dir / f'speed_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "speed,speed", #m/s
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "8",
                "--min-width-value", "8",
                "--max-color-value", "10",
                "--max-width-value", "10",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )

# Example-5: A coloured city map providing Relative Speed Analysis - This calculates the ratio between the average simulated speed and the legal speed limit on each road. A ratio significantly below 1.0 indicates congestion, providing key insight into the model's functionality and performance.

        output_path = plots_dir / f'speedRelative_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "speed,speed",
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "0.9",
                "--min-width-value", "0.9",                
                "--max-color-value", "1",
                "--max-width-value", "1",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )

# Example-6: Teleportation analysis - number of vehicles teleported on roads during simulation
# To understand realism of simulation environment, teleportation is not desired.

        output_path = plots_dir / f'teleported_{date_str}_{f.stem}.png'
        run_plot_net_dump(
            net_xml,
            f,
            "speed,speed",
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "0",
                "--max-color-value", "15",
                "--max-width-value", "15",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "jet"
            ]
        )
# ---------------------------------------------
# Section 6 — SUMO Visualization PART3: Compare different simulation outputs
# ---------------------------------------------

# Compare multiple simulation outputs - real vs augmented data generated model simulations
# This allows comparison of the same metric across different scenarios
# Scenarios are comparable if real vs augmented outputs show similar patterns

def run_plot_net_dump_comparison(netfile, edgefiles, measures, labels, out, extra_args=None):
    """Call SUMO's plot_net_dump.py to compare multiple files."""
    # Check if input files exist
    if not netfile or not netfile.exists():
        print(f"Network file {netfile} not found. Skipping comparison.")
        return None
        
    for edgefile in edgefiles:
        if not edgefile.exists():
            print(f"Edge data file {edgefile} not found. Skipping comparison.")
            return None
    
    script = shutil.which('plot_net_dump.py')
    if not script and PLOT_TOOLS_DIR:
        candidate = PLOT_TOOLS_DIR / 'plot_net_dump.py'
        if candidate.exists():
            script = str(candidate)
    if not script:
        print('plot_net_dump.py not found. Please set SUMO_HOME or install SUMO tools.')
        return None
    
    # Build input argument with multiple files
    input_arg = ",".join([str(f) for f in edgefiles])
    
    cmd = [script, "-v", "-n", str(netfile), "--measures", measures, "-i", input_arg, "--output", out]
    
    # Add labels if provided
    if labels:
        cmd.extend(["--labels", ",".join(labels)])
    
    if extra_args:
        cmd += extra_args
    
    print("Running comparison:", " ".join(cmd))
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=180)  # Longer timeout for comparisons
        if result.returncode != 0:
            print(f"Error running plot_net_dump comparison: {result.stderr}")
            return None
        return out
    except subprocess.TimeoutExpired:
        print(f"Comparison plotting timed out")
        return None
    except Exception as e:
        print(f"Unexpected error in comparison plotting: {e}")
        return None

# Group edgeoutput files by date for comparison
def find_matching_files_by_date(real_files, augmented_files):
    """Find matching real and augmented files by date."""
    matches = []
    
    # Extract dates from real files
    real_dates = {}
    for f in real_files:
        if 'edgedata.out' in f.name:
            date_str = extract_date_from_path(f)
            if date_str != "unknown_date":
                real_dates[date_str] = f
    
    # Find matching augmented files
    for f in augmented_files:
        if 'edgedata.out' in f.name:
            date_str = extract_date_from_path(f)
            if date_str in real_dates:
                matches.append((real_dates[date_str], f, date_str))
    
    return matches

print("\n" + "="*60)
print("Section 6 - Comparing Real vs Augmented Outputs")
print("="*60)

# Find matching files by date
matching_pairs = find_matching_files_by_date(real_output_files, augmented_output_files)

if matching_pairs:
    print(f"Found {len(matching_pairs)} date-matched file pairs for comparison")
    
    for real_file, aug_file, date_str in matching_pairs:
        print(f"\nComparing files for date: {date_str}")
        print(f"  Real: {real_file.name}")
        print(f"  Augmented: {aug_file.name}")
        
        files_to_compare = [real_file, aug_file]
        labels = [f"Real_{date_str}", f"Augmented_{date_str}"]
        
        # Comparison 1: A coloured city map showing vehicle movement/edge usage differences between scenarios for the same day.
        
        output_path = plots_dir / f'comparison_entered_{date_str}.png'
        run_plot_net_dump_comparison(
            net_xml,
            files_to_compare,
            "entered,entered",
            labels,
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "-100",
                "--min-width-value", "-100",            
                "--max-color-value", "100",
                "--max-width-value", "100",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "#0:#0000c0,.25:#404080,.5:#808080,.75:#804040,1:#c00000",
                "--legend-loc", "upper right"
            ]
        )

        # Comparison 2: A coloured city map showing time loss differences between scenarios for the same day.
        
        output_path = plots_dir / f'comparison_timeloss_{date_str}.png'
        run_plot_net_dump_comparison(
            net_xml,
            files_to_compare,
            "timeLoss,timeLoss",
            labels,
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "-300",
                "--min-width-value", "-300",            
                "--max-color-value", "300",  #5min
                "--max-width-value", "300",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "#0:#0000c0,.25:#404080,.5:#808080,.75:#804040,1:#c00000", 
                "--legend-loc", "upper right"
            ]
        )
        
        # Comparison 3: A coloured city map showing mean speed differences between scenarios for the same day.
        
        output_path = plots_dir / f'comparison_speed_{date_str}.png'
        run_plot_net_dump_comparison(
            net_xml,
            files_to_compare,
            "speed,speed",
            labels,
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "-0.5",
                "--min-width-value", "-0.5",            
                "--max-color-value", "0.5",
                "--max-width-value", "0.5",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "#0:#0000c0,.25:#404080,.5:#808080,.75:#804040,1:#c00000",
                "--legend-loc", "upper right"
            ]
        )
        
        # Comparison 4: A coloured city map showing vehicle teleportation differences between scenarios for the same day.
        output_path = plots_dir / f'comparison_teleported_{date_str}.png'
        run_plot_net_dump_comparison(
            net_xml,
            files_to_compare,
            "teleported,teleported",
            labels,
            str(output_path),
            extra_args=[
                "--xlim", "900,16000",
                "--ylim", "4500,16000",
                "--xticks", "1000,16000,2500,5",
                "--yticks", "2000,16000,2500,5",
                "--xlabel", "[m]",
                "--ylabel", "[m]",
                "--default-width", "0.3",
                "--default-color", "#606060",
                "--min-color-value", "-10",
                "--min-width-value", "-10",            
                "--max-color-value", "10",
                "--max-width-value", "10",
                "--max-width", "1.3",
                "--min-width", "0.3",
                "--colormap", "#0:#0000c0,.25:#404080,.5:#808080,.75:#804040,1:#c00000",
                "--legend-loc", "upper right"
            ]
        )
else:
    print("No matching date pairs found for comparison between real and augmented outputs")

# Summary output
print("\n" + "="*50)
print("Final Summary:")
print(f"Plots directory: {plots_dir}")
print(f"Total real output files found: {len(real_output_files)}")
print(f"Total augmented output files found: {len(augmented_output_files)}")
print(f"Date-matched pairs for comparison: {len(matching_pairs)}")

# Count files by type and show dates found
file_types = {}
dates_found = set()

for f in real_output_files:
    date_str = extract_date_from_path(f)
    dates_found.add(date_str)
    
    for file_type in ['vehroute', 'summary', 'tripinfo', 'detector.out', 'edgedata.out', 'fcd']:
        if file_type in f.name:
            if file_type not in file_types:
                file_types[file_type] = 0
            file_types[file_type] += 1
            break

print("Files by type:")
for file_type, count in file_types.items():
    print(f"  {file_type}: {count}")

print("Dates found in file paths:")
for date in sorted(dates_found):
    print(f"  {date}")

print("Plot generation completed!")
print("="*50)